### r3bmodel

In [ ]:
#data = genfromtxt('raw_data_test.txt', delimiter=',')
#data = genfromtxt('validation_raw.txt',delimiter=',')
data = genfromtxt('data_raw_uniform_test.csv',delimiter=',')
#data = genfromtxt('data_stream_2121.txt', delimiter=',')
#my_data[:,4] = my_data[:,4]+4500  #this step is needed, I only want positive time values, so that I can use the time as a radius
data[:,4] = 1  #this step is needed, I only want positive time values, so that I can use the time as a radius
# ### structure of mydata : eventnr, energy, theta, phi, hit-time
data = data*[1.,1.,3.14159/180,3.14159/180,1.]
unique_values, continuous_numbers = np.unique(data[:,0], return_inverse=True)
data[:,0] = continuous_numbers
print("continuous numbers:", continuous_numbers)
out_cluster_r3b = run_r3b_clustering(data,0.25)
#foo = plot_true_reco(data)
foo_comb = get_true_combinatorics(data)
foo_ene = get_true_reco_ene(data,out_cluster_r3b,foo_comb)

### agglomerative model

In [ ]:
#data = genfromtxt('raw_data_test.txt', delimiter=',')
data = genfromtxt('data_raw_uniform_test.csv', delimiter=',')
#data = genfromtxt('data_stream_2121.txt', delimiter=',')
data[:,4] = data[:,4]+4500  #this step is needed, I only want positive time values, so that I can use the time as a radius
#my_data[:,4] = 1  #this step is needed, I only want positive time values, so that I can use the time as a radius

# ### structure of mydata : eventnr, energy, theta, phi, hit-time
data = data*[1.,1.,3.14159/180,3.14159/180,1.]
unique_values, continuous_numbers = np.unique(data[:,0], return_inverse=True)
data[:,0] = continuous_numbers
out_cluster_agglo = run_agglo_clustering(data,3540)
#foo = plot_true_reco(data)
foo_comb = get_true_combinatorics(data)
print("data size:", len(out_cluster_agglo))
print("true_size:",len(foo_comb))
print("type of out_cluster_agglo:",type(out_cluster_agglo))
foo_ene = get_true_reco_ene(data,out_cluster_agglo,foo_comb)

### edge model

#### training data

In [ ]:
#training_data = genfromtxt('training_all_output_false_negative.txt', delimiter=',')
#training_data = genfromtxt('training_raw.txt', delimiter=',')
##training on raw data with uniform energy distribution between 0.3 MeV and 10 MeV
training_data = genfromtxt('data_raw_uniform_training.csv', delimiter=',')
## some data manipulation needed
training_data[:,4] =(training_data[:,4]-np.min(training_data[:,4]))/(np.max(training_data[:,4])-np.min(training_data[:,4]))
unique_values, continuous_numbers = np.unique(training_data[:,0], return_inverse=True)
training_data[:,0] = continuous_numbers
training_data[:,2] = training_data[:,2]*math.pi/180.
training_data[:,3] = training_data[:,3]*math.pi/180.
training_data[:,4] = 1  ## set time to 1, not sensitive to time, TODO: remove later, just a test
training_true = get_true_combinatorics(training_data)
input_training_data = manip_data_ff(training_data,12) ##second param gives number of features

pred_training = train_two_hidden_model(input_training_data,torch.FloatTensor(training_true),12,1000,5e-3,100)

training_reco = get_true_reco_ene(training_data,pred_training[0].tolist(),training_true)

#### validation data

In [ ]:
#val_data = genfromtxt('validation_all_output_false_negative.txt', delimiter=',')
#val_data = genfromtxt('validation_raw.txt', delimiter=',')
##test on raw data with uniform energy distribution from 0.3MeV to 10MeV
val_data = genfromtxt('data_raw_uniform_test.csv', delimiter=',')
## some data manipulation needed
val_data[:,4] =(val_data[:,4]-np.min(val_data[:,4]))/(np.max(val_data[:,4])-np.min(val_data[:,4]))
unique_values, continuous_numbers = np.unique(val_data[:,0], return_inverse=True)
val_data[:,0] = continuous_numbers
val_data[:,2] = val_data[:,2]*math.pi/180.
val_data[:,3] = val_data[:,3]*math.pi/180.
val_data[:,4] = 1 ## set time to 1, not sensitive to time, TODO: remove later, just a test
val_true = get_true_combinatorics(val_data)
input_val_data = manip_data_ff(val_data,12) ##second param gives number of features

pred_val = validate_two_hidden_model(input_val_data,torch.FloatTensor(val_true),12,1000,5e-3,100)

val_reco = get_true_reco_ene(val_data,pred_val[0].tolist(),val_true)

#### just test to substract bin count

In [ ]:
hist1, bins1 = np.histogram(val_reco[0], bins=np.linspace(0, 10, 100))
hist2, bins2 = np.histogram(val_reco[1], bins=np.linspace(0, 10, 100))
residual = (hist1 - hist2)
a = np.arange(0,9.9,0.1)
print(a)
print(a.shape)
print(residual.shape)
plt.hist(a,100, weights=residual)